In [1]:
import os
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from py_light.Dataset import QADataset
from torchvision import transforms

test_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize((128,128)),
        transforms.Normalize([0.7760, 0.7491, 0.7213], [0.2949, 0.3032, 0.3314]),
    ]
)
# For training, we add some augmentation. Networks are too powerful and would overfit.
train_transform = transforms.Compose(
    [
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(),
        # transforms.RandomResizedCrop((32, 32), scale=(0.8, 1.0), ratio=(0.9, 1.1)),
        transforms.ToTensor(),
        transforms.Normalize([0.7760, 0.7491, 0.7213], [0.2949, 0.3032, 0.3314]),
    ]
)

dir = os.path.join(os.getcwd(),"Dataset/Train/Image")
train_dataset = QADataset(transform=train_transform, loc = dir)
print(len(train_dataset))
val_dataset = QADataset(transform = test_transform, loc = dir, istrain =  False)
print(len(val_dataset))

from torch.utils.data import DataLoader
# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=24, shuffle=True) 
val_loader = DataLoader(train_dataset, batch_size=48)

c:\ProgramData\anaconda3\envs\transformer\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['c:\\Workspace\\Causal_Inference\\Dataset/Train/Image\\Achievement\\004436'
 'c:\\Workspace\\Causal_Inference\\Dataset/Train/Image\\Achievement\\004547'
 'c:\\Workspace\\Causal_Inference\\Dataset/Train/Image\\Achievement\\004567'
 ...
 'c:\\Workspace\\Causal_Inference\\Dataset/Train/Image\\Processing\\302093'
 'c:\\Workspace\\Causal_Inference\\Dataset/Train/Image\\Processing\\302097'
 'c:\\Workspace\\Causal_Inference\\Dataset/Train/Image\\Processing\\302098']
48000
6000


In [2]:
import torch
from py_light.ViT import ViT_trans
ImageFile.LOAD_TRUNCATED_IMAGES = True
#torch.set_float32_matmul_precision('high')
# 모델 인스턴스 생성
model_kwargs = {
    'embed_dim': 128,
    'hidden_dim': 512,
    'num_channels': 3,
    'num_heads': 8,
    'num_layers': 6,
    'num_classes': 3,
    'patch_size': 16,
    'num_patches': 64,
    'dropout': 0.1,
    'head_num_layers': 2 
}

model = ViT_trans(model_kwargs, lr=1e-3)

# 트레이너 설정 및 학습

# 사용 가능한 장치 확인
if torch.backends.mps.is_available():
    # MPS 사용 가능한 경우
    device = torch.device("mps")
    accelerator = 'mps'
elif torch.cuda.is_available():
    # CUDA GPU 사용 가능한 경우
    device = torch.device("cuda")
    accelerator = 'gpu'
else:
    # CPU 사용
    device = torch.device("cpu")
    accelerator = 'cpu'
    
trainer = pl.Trainer(max_epochs=2, accelerator=accelerator, devices=1)

trainer.fit(model, train_loader, val_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type              | Params
---------------------------------------------------
0 | model        | VisionTransformer | 1.3 M 
1 | dropout      | Dropout           | 0     
2 | transformer  | Sequential        | 396 K 
3 | mlp_head     | Sequential        | 643   
4 | f1_cal       | MulticlassF1Score | 0     
  | other params | n/a               | 768   
---------------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.780     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\ProgramData\anaconda3\envs\transformer\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00,  5.49it/s]

c:\ProgramData\anaconda3\envs\transformer\Lib\site-packages\pytorch_lightning\utilities\data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 48. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


c:\ProgramData\anaconda3\envs\transformer\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Epoch 0:   1%|          | 14/2000 [00:32<1:17:50,  0.43it/s, v_num=2]

OSError: image file is truncated (93 bytes not processed)